In [36]:
import PyPDF2 as p2
#from pdfminer.pdfparser import PDFParser
#from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pickle
from bs4 import BeautifulSoup

import json
import numpy as np
import re

import fitz

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [37]:
df_outlines = pd.read_csv("F:\Environmental Baseline Data\Version 4 - Final\Indices\Index 2 - PDFs for Major Projects with ESAs - old.csv")
len(df_outlines)
df_outlines[20:30].head(5)
dataIDs = df_outlines['DataID']
len(dataIDs)

1890

,file_name,application_name,esa_folder_link,DocumentID,DataID,esa_download_link,DataID_pdf,Section,Application filing date,link to folder with ESA,...,Regulatory Instruments Issued,Pipeline Named in GIS,Application title short,Link to Application,Link to Decision,Location,Component,Component PDF Name,page_count,size
20,C03258-3 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3J7,3891472,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3891472.pdf,52,2004-10-07,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,...,GC-116\n,Mackenzie Valley Gas Pipeline,Application for the construction and operation...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN,NaN,NaN,NaN,NaN
21,C03258-4 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3J8,3891895,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3891895.pdf,52,2004-10-07,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,...,GC-116\n,Mackenzie Valley Gas Pipeline,Application for the construction and operation...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN,NaN,NaN,NaN,NaN
22,C03258-5 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3J9,3891896,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3891896.pdf,52,2004-10-07,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,...,GC-116\n,Mackenzie Valley Gas Pipeline,Application for the construction and operation...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN,NaN,NaN,NaN,NaN
23,C03258-6 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3K0,3892113,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3892113.pdf,52,2004-10-07,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,...,GC-116\n,Mackenzie Valley Gas Pipeline,Application for the construction and operation...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN,NaN,NaN,NaN,NaN
24,C03258-7 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3K1,3891801,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3891801.pdf,52,2004-10-07,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,...,GC-116\n,Mackenzie Valley Gas Pipeline,Application for the construction and operation...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN,NaN,NaN,NaN,NaN


1890

In [38]:
def area_of_imgblocks(imgblocks):
    sum_areas = 0
    for imgblock in imgblocks:
        block_area = imgblock['width']* imgblock['height']
        sum_areas = sum_areas + block_area
    return(sum_areas)

In [39]:
def extract_image_features(dataIDs):
    dataID_list = []
    pageNo = []
    No_of_images = []
    Area_of_images = []
    Percent_images = []
    
    for dataID in dataIDs:
        pdf_path = 'F:/Environmental Baseline Data/Version 4 - Final/PDF/' + str(dataID) +'.pdf'

        doc = fitz.open(pdf_path)
        img_ = dict()
        page_no = 0
        
    
        for page in doc:  # iterate through the pages
            page_no = page_no + 1
            d = page.getText("dict")
            blocks = d["blocks"]
            imgblocks = [b for b in blocks if b["type"] == 1]
            pageNo.append(page_no)
            No_of_images.append(len(imgblocks))
            Area_of_images.append(area_of_imgblocks(imgblocks))
            dataID_list.append(dataID)
        
    return No_of_images, Area_of_images, dataID_list, pageNo

In [40]:
No_of_images, Area_of_images, dataID_list, pageNo = extract_image_features(dataIDs)

mupdf: cannot find page 3363 in page tree
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invali

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

In [42]:
Xdf_dataset = pd.DataFrame({'DataID' : dataID_list,
                            'PageNo': pageNo,
                            'No_of_images' : No_of_images, 
                            'Area_of_images' : Area_of_images
                           })
len(Xdf_dataset)
Xdf_dataset.head()

83555

,DataID,PageNo,No_of_images,Area_of_images
0,268706,1,1,79376
1,268706,2,1,79376
2,268706,3,1,79376
3,268706,4,1,79376
4,268706,5,1,79376


In [43]:
previous_pdf = 0
#pdf_name = []
mean_area = {}
page_numbers = {}
sum_area = 0
count_pdf = 0

# Get Mean arae for each PDF files 
for i in range(len(Xdf_dataset)):
    current_pdf = Xdf_dataset.iloc[i].DataID
    page_no = Xdf_dataset.iloc[i].PageNo
    
    if previous_pdf != current_pdf:
        if previous_pdf != 0:
            if sum_area == 0:
                sum_area = 1
            mean_area[previous_pdf] = sum_area/count_pdf
            page_numbers[previous_pdf] = count_pdf
            sum_area = 0
            count_pdf = 0
        previous_pdf = current_pdf
    
    sum_area = sum_area + Xdf_dataset.iloc[i].Area_of_images
    count_pdf =count_pdf +1
    
mean_area[previous_pdf] = sum_area/count_pdf
page_numbers[previous_pdf] = count_pdf

len(mean_area)
len(page_numbers)
#len(pdf_name)
#len(DataIDHand)
mean_area

1887

1887

{268706: 76385.04672897197,
 268709: 61852.51282051282,
 268712: 48444.148148148146,
 269018: 94439.92857142857,
 269021: 683986.8666666667,
 269024: 79143.5,
 269027: 79376.0,
 269030: 79376.0,
 269033: 882655.5714285715,
 268930: 862609.074074074,
 269036: 79376.0,
 269039: 79376.0,
 268933: 387623.5172413793,
 269042: 79376.0,
 268936: 79356.92307692308,
 269045: 79376.0,
 268939: 79376.0,
 269048: 79163.42857142857,
 3892112: 659726.5411764706,
 3892242: 194797.4484848485,
 3891472: 1607411.8504672898,
 3891895: 1580307.936,
 3891896: 275560.7105263158,
 3892113: 357998.4072727273,
 3891801: 1494868.6746987952,
 3892114: 2742522.304812834,
 3891473: 345390.3945945946,
 3892358: 94489.28095238096,
 3891802: 3444579.705882353,
 3892115: 577243.8944099379,
 3892116: 802697.0486891386,
 3892458: 519588.7438016529,
 3892459: 202995.14285714287,
 3892359: 123706.19069767442,
 3892360: 115525.35820895522,
 3891803: 62056.09656652361,
 3891804: 1172596.254826255,
 3891805: 311.538461538461

In [44]:
area_ratio = []
for i in range(len(Xdf_dataset)):
    current_pdf = Xdf_dataset.iloc[i].DataID
    page_no = Xdf_dataset.iloc[i].PageNo
    area_ratio.append(Xdf_dataset.iloc[i].Area_of_images/mean_area[current_pdf])
len(area_ratio)
#len(page_numbers)
Xdf_dataset['area_ratio'] = area_ratio
Xdf_dataset.head(20)

83555

,DataID,PageNo,No_of_images,Area_of_images,area_ratio
0,268706,1,1,79376,1.039156
1,268706,2,1,79376,1.039156
2,268706,3,1,79376,1.039156
3,268706,4,1,79376,1.039156
4,268706,5,1,79376,1.039156
5,268706,6,1,79376,1.039156
6,268706,7,1,79376,1.039156
7,268706,8,1,79376,1.039156
8,268706,9,1,79376,1.039156
9,268706,10,1,79376,1.039156


In [45]:
Xdf_dataset.to_csv("F:/Environmental Baseline Data/Version 4 - Final/Support files/Random Forest/Image_Area_Data.csv")

In [48]:
len(set(Xdf_dataset['DataID']))
len(dataIDs)

1887

1890

In [49]:
for udataID in dataIDs:
    if udataID not in set(Xdf_dataset['DataID']):
        print(udataID)